In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [ ]:
CHROME_DRIVER_PATH="D:\\00-env\\chromedriver.exe"
YOUTUBE_LINKS = {
    'oh_my_girl': {
        'nonstop' : 'https://www.youtube.com/watch?v=iDjQSdN_ig8',
        'the_fifth_season' : 'https://www.youtube.com/watch?v=udGwca1HBM4',
        'bungee' : 'https://www.youtube.com/watch?v=QTD_yleCK9Y',
    },
    'chungha': {
        'stay_tonight': 'https://www.youtube.com/watch?v=YPFIh0dfYfw',
        'roller_coaster': 'https://www.youtube.com/watch?v=900X9fDFLc4',
        'snapping' : 'https://www.youtube.com/watch?v=deV_DmHKwjc'
    }
}

all_comments = dict()


In [ ]:
def start_webpage(driver,link):
    driver.get(link)
    time.sleep(3)
    driver.execute_script("window.scrollTo(1, 500);")
    
def scrollDownToEnd(driver, pause_time=0.5):
    while True:
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script("window.scrollTo(1, document.documentElement.scrollHeight);")

        time.sleep(pause_time)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            driver.execute_script("window.scrollTo(1, document.documentElement.scrollHeight);")
            time.sleep(pause_time)

            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            else:
                last_height = new_height
                continue

def print_dict(origin_key, d):
    for k, v in d.items():
        if isinstance(v, dict):
            print_dict(k, v)
        else:
            print(origin_key, k, v)

def get_all_comments(driver):
    comment_div= driver.find_element_by_xpath('//*[@id="contents"]')
    contents = comment_div.find_elements_by_xpath('//*[@id="content-text"]')
    authors = comment_div.find_elements_by_xpath('//*[@id="author-text"]')

    all_comments = list()
    
    print("comments: ", len(contents))
    
    for idx, content in enumerate(contents):
        comment = dict()
        comment[authors[idx].text] = content.text
        all_comments.append(comment)

    return all_comments

def automatic_crawlling(driver, singer, title, link):
    print(singer, title, link)
    start_webpage(driver, link)
    
    time.sleep(3)
    
    comments = get_all_comments(driver)
    
    if singer not in all_comments.keys():
        all_comments[singer] = dict()
        
    all_comments[singer][title] = comments

def sequential_start(driver, d, origin_key=None):
    for k, v in d.items():
        if isinstance(v, dict):
            sequential_start(driver, v, k)
        else:
            automatic_crawlling(driver, origin_key, k, v)

In [ ]:
driver = webdriver.Chrome(CHROME_DRIVER_PATH)
sequential_start(YOUTUBE_LINKS)